In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/artificial-neural-networks-and-deep-learning-2020.zip

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
import os
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
apply_data_augmentation = False

In [8]:
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255, 
                                        validation_split=0.25)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

In [9]:

test_data_gen = ImageDataGenerator(rescale=1./255)

In [10]:
dataset_dir = os.path.join(cwd, 'MaskDataset')

In [11]:
dataset_dir

'/content/MaskDataset'

In [12]:
bs = 16

In [13]:
img_h = 256
img_w = 256

In [14]:
num_classes=3


In [15]:
import pandas as pd
import numpy as np
label_path = os.path.join(dataset_dir, 'train_gt.json')
labels = pd.read_json(label_path, orient='index')
labels = labels.reset_index()
train_labels = labels.rename(columns={"index": "id", 0: "label"})
train_labels['label'] = train_labels['label'].astype(str)
train_labels

,id,label
0,14985.jpg,0
1,13358.jpg,0
2,10210.jpg,0
3,18202.jpg,0
4,14962.jpg,0
...,...,...
5609,14383.jpg,2
5610,11787.jpg,2
5611,15770.jpg,2
5612,10934.jpg,2


In [16]:
img_h

256

In [17]:
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_dataframe(dataframe=train_labels,
                                               directory=training_dir,
                                               x_col="id",
                                               y_col="label",
                                               subset="training",
                                              #  batch_size=bs,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  # targets are directly converted into one-hot vectors


Found 4211 validated image filenames belonging to 3 classes.


In [18]:
test_files = pd.DataFrame()
test_files['file'] = os.listdir('/content/MaskDataset/test')

In [19]:
training_dir = os.path.join(dataset_dir, 'training')
valid_gen = train_data_gen.flow_from_dataframe(dataframe=train_labels,
                                               directory=training_dir,
                                               x_col="id",
                                               y_col="label",
                                               subset="validation",
                                              #  batch_size=bs,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)

Found 1403 validated image filenames belonging to 3 classes.


In [20]:
test_dir = os.path.join(dataset_dir, 'test')
test_gen = train_data_gen.flow_from_dataframe(dataframe=test_files,
                                              directory=test_dir,
                                               x_col='file',
                                               y_col=None,
                                               class_mode=None,
                                               color_mode='rgb',
                                               shuffle=False,
                                               seed=SEED)

Found 450 validated image filenames.


In [21]:
train_gen[0]

(array([[[[0.89019614, 0.9294118 , 0.8980393 ],
          [0.86274517, 0.94117653, 0.73333335],
          [0.69411767, 0.7294118 , 0.5137255 ],
          ...,
          [0.49803925, 0.5019608 , 0.13725491],
          [0.5529412 , 0.5686275 , 0.2392157 ],
          [0.5529412 , 0.5686275 , 0.2392157 ]],
 
         [[0.75294125, 0.73333335, 0.58431375],
          [0.77647066, 0.7960785 , 0.54509807],
          [0.59607846, 0.58431375, 0.36862746],
          ...,
          [0.6509804 , 0.6666667 , 0.3254902 ],
          [0.6862745 , 0.7137255 , 0.40000004],
          [0.67058825, 0.69803923, 0.3921569 ]],
 
         [[0.6156863 , 0.60784316, 0.37647063],
          [0.5921569 , 0.6117647 , 0.3529412 ],
          [0.20784315, 0.20000002, 0.        ],
          ...,
          [0.7019608 , 0.73333335, 0.40784317],
          [0.7686275 , 0.7960785 , 0.49411768],
          [0.7490196 , 0.7725491 , 0.48235297]],
 
         ...,
 
         [[0.7607844 , 0.6784314 , 0.6039216 ],
          [0.47450

In [22]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))


valid_dataset = valid_dataset.repeat()


In [23]:
# Architecture: Features extraction -> Classifier

start_f = 8
depth = 8

model = tf.keras.Sequential()

# Features extraction
for i in range(depth):

    if i == 0:
        input_shape = [img_h, img_w, 3]
    else:
        input_shape=[None]

    # Conv block: Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=input_shape))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

    start_f *= 2
    
# Classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [24]:
model.summary()

# Visualize initialized weights
model.weights

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 8)       224       
_________________________________________________________________
re_lu (ReLU)                 (None, 256, 256, 8)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 16)      1168      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 128, 128, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        4

[<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 3, 8) dtype=float32, numpy=
 array([[[[ 1.87292695e-02, -6.66571707e-02,  4.01918888e-02,
           -1.26130164e-01, -3.43003571e-02,  1.99994743e-01,
            3.55525315e-03, -6.95133060e-02],
          [ 1.37295932e-01,  4.19998467e-02,  1.32354498e-01,
           -8.24365020e-03,  4.30276692e-02, -2.35006183e-01,
           -3.16811651e-02,  2.35468864e-01],
          [ 2.36862689e-01, -1.74060702e-01,  2.43510842e-01,
            1.38533324e-01, -1.37798309e-01, -5.41950762e-03,
            6.55457079e-02,  3.13819945e-02]],
 
         [[-4.74313945e-02, -4.98768538e-02,  8.99522901e-02,
            1.34971857e-01, -2.68238485e-02, -2.18192115e-01,
           -1.25084400e-01, -9.58562642e-02],
          [-8.99255723e-02, -1.51148498e-01,  1.60095960e-01,
            4.54548597e-02, -6.18434399e-02,  1.69046938e-01,
           -1.27805948e-01,  1.58601552e-01],
          [-1.66178837e-01, -1.14610642e-01, -4.28984761e-02,
            

In [29]:
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 3e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [30]:
import os
from datetime import datetime


cwd = os.getcwd()

exps_dir = os.path.join('/content/drive/My Drive/Keras3/', 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks.append(es_callback)


In [31]:
callbacks

In [37]:
model.fit(x=train_dataset,
          epochs=20, 
          verbose=1,
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Epoch 1/20
132/132 [==============================] - 478s 4s/step - loss: 0.1189 - accuracy: 0.9584 - val_loss: 4.3595 - val_accuracy: 0.4113
Epoch 2/20
132/132 [==============================] - 479s 4s/step - loss: 0.0845 - accuracy: 0.9717 - val_loss: 3.4980 - val_accuracy: 0.5253
Epoch 3/20
132/132 [==============================] - 480s 4s/step - loss: 0.0726 - accuracy: 0.9736 - val_loss: 4.5624 - val_accuracy: 0.3927
Epoch 4/20
132/132 [==============================] - 475s 4s/step - loss: 0.0739 - accuracy: 0.9765 - val_loss: 5.4040 - val_accuracy: 0.2680
Epoch 5/20
132/132 [==============================] - 464s 4s/step - loss: 0.0530 - accuracy: 0.9843 - val_loss: 3.0587 - val_accuracy: 0.5445
Epoch 6/20
132/132 [==============================] - 490s 4s/step - loss: 0.0524 - accuracy: 0.9820 - val_loss: 4.3280 - val_accuracy: 0.4184
Epoch 7/20
132/132 [==============================] - 486s 4s/step - loss: 0.0323 - accuracy: 0.9903 - val_loss: 5.6926 - val_accuracy: 0.3514

In [38]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

Y_prediction = model.predict_generator(test_gen, len(test_gen))

Y_pred = np.argmax(Y_prediction,axis = 1) 


prediction_df = pd.DataFrame({'Id': test_gen.filenames, 'Category': Y_pred})

In [39]:
prediction_df

,Id,Category
0,16278.jpg,0
1,14391.jpg,1
2,15017.jpg,1
3,14113.jpg,0
4,13911.jpg,2
...,...,...
445,14230.jpg,1
446,10581.jpg,1
447,13100.jpg,2
448,12448.jpg,2


In [40]:
prediction_df.to_csv('prediction.csv')

In [ ]:
# Run this cell only if you want to plot the confusion matrix in tensorboard
import io
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

file_writer_cm = tf.summary.create_file_writer(tb_dir + '/cm')

# Function to convert input figure to png tf image 
# (for plotting the confusion matrix in tensorboard)
def plot_to_image(figure):
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

# Function to compute the confusion matrix (using sklearn) and to save it 
# in tensorboard
def log_confusion_matrix(epoch, logs):
  Y_prediction = model.predict_generator(test_gen, len(test_gen))
  # Convert predictions classes to one hot vectors 
  Y_pred_classes = np.argmax(Y_prediction,axis = 1) 
  # Convert validation observations to one hot vectors
  Y_true = test_gen.classes
  # compute the confusion matrix
  confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
  fig = plt.figure(figsize=(10,8))
  sns.heatmap(confusion_mtx, annot=True, fmt="d");
  
  cm_image = plot_to_image(fig)

  with file_writer_cm.as_default():
    tf.summary.image("Confusion Matrix", cm_image, step=epoch)

cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)
callbacks.append(cm_callback)

In [ ]:
%tensorboard --logdir /content/drive/My\ Drive/Keras3/classification_experiments/